# LCT-AGI

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/langchain-tools/blob/main/examples/sample.ipynb)

In [1]:
%%capture
!pip install langchain
!pip install openai

In [114]:
pip list

Package                       Version
----------------------------- -----------
aiofiles                      22.1.0
aiohttp                       3.8.4
aiosignal                     1.3.1
aiosqlite                     0.18.0
alembic                       1.10.2
altair                        4.2.2
anyio                         3.6.2
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
asttokens                     2.2.1
async-generator               1.10
async-timeout                 4.0.2
attrs                         22.2.0
Babel                         2.12.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
beautifulsoup4                4.11.2
bleach                        6.0.0
blinker                       1.5
bokeh                         2.4.3
Bottleneck                    1.3.7
brotlipy                      0.7.0
cached-property               1.5.2
certifi                       2022.12.7
certipy                       0.1.3
cffi       

In [1]:
import os

from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, tool
from langchain.tools import BaseTool

In [3]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"


### サンプル

In [49]:
class MultiplicationTool(BaseTool):
    name = "MultiplicationTool"
    description = """used for multiplication. The input is two numbers. For example, if you want to multiply 1 by 2, the input is '1,2'."""

    def _run(self, query: str) -> str:
      """Use the tool."""
      a, b = query.split(",")
      c = int(a) * int(b)
      result = c

      return result 

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("BingSearchRun does not support async")

In [50]:
tools = [MultiplicationTool()]

llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [51]:
tools[0].description

"used for multiplication. The input is two numbers. For example, if you want to multiply 1 by 2, the input is '1,2'."

In [52]:
agent.run('38 times 47?')



> Entering new AgentExecutor chain...
 I need to multiply two numbers
Action: MultiplicationTool
Action Input: 38,47
Observation: 1786
Thought: I now know the final answer
Final Answer: 38 times 47 is 1786.

> Finished chain.


'38 times 47 is 1786.'

In [ ]:
llm('38 times 47?')

### 参照するLLM

In [8]:
#input = 'trainig XGBoost'
input = 'training LightGBM'
exec_list = [
    'training LightGBM'
]

In [9]:
def deciding_agent(input_, exec_list_):
    print("deciding agent")
    
    decide_llm = OpenAI(temperature=0,model_name="gpt-3.5-turbo")

    i = ''
    for i in exec_list:
        i = i + ','
    i = '[' + i + ']'

    prompt = """
    You are an agent that determines if the input task is executable. 
    All you can do is to be included in {exec_list}. 
    Answer "Yes." if you can perform the task, or "No." if you cannot.
    ------
    The entered task is:{input}
    ------
    """.format(exec_list = i,input = input_)

    return decide_llm(prompt)

In [10]:
deciding_agent(input, exec_list)

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:165: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:672: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


'Yes'

### 道具をつくるLLM

In [15]:
def toolmaking_agent(input_):
    print("toolmaking agent")

    toolmaking_llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo")


    prompt = """
    Create a python function that can execute {input} with a single string as input.
    The code below is the code created with the input "multiply two numbers".
    The output is only the code of the created python.
    ```
    from dataclasses import dataclass

    @dataclass
    class NewTool(BaseTool):
        name = "MultiplicationTool"
        description = "used for multiplication. The input is two numbers. For example, if you want to multiply 1 by 2, the input is '1,2'."

        def _run(self, query: str) -> str:
            "Use the tool."
            a, b = query.split(",")
            c = int(a) * int(b)
            result = c

        return result 

        async def _arun(self, query: str) -> str:
            "Use the tool asynchronously."
            raise NotImplementedError("BingSearchRun does not support async")
    ```
    """.format(input = input_)

    code = toolmaking_llm(prompt)


    file = open('/content/new_tool.py', mode='w')
    file.write(code)
    file.close()

    return code

In [24]:
input = 'Output multiples of 3 up to 20'
toolmaking_agent(input)

toolmaking agent


'finish'

### 実行するLLM

In [ ]:
def executing_agent(input_, tools_):
    print("executing agent")

    excute_llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo")

    agent = initialize_agent(tools_, excute_llm, agent="zero-shot-react-description", verbose=True)

    agent.run(input_)

    return

### 全体

In [ ]:
def main(input_, exec_list_, tools_):
    
    output = deciding_agent(input_, exec_list_)

    if output == "Yes":
        print('Execute it as it is executable without creating a tool.')
        executing_agent(input_, tools_)

    elif output == "No":
        print('It is necessary to create a tool, so run it after creating the tool.')
        new_tool = toolmaking_agent(input_)

        exec(new_tool)

        tools_.append(NewTool())

        executing_agent(input_, tools_)
    
    return 

In [40]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, tool

In [107]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, tool


class LCTAGI():
    def __init__(
        self,
        model_name = 'gpt-4'
        ):
        self.model_name = model_name

    
    def _generalize(self, input_prompt_):

        generalize_llm = OpenAI(temperature=0,model_name = self.model_name)

        prompt = """
        Please generalize the following sentences by replacing any numerical or other parts of the sentences with letters.
        Output is only the text after rewriting.

        ------------
        {input}
        ------------
        """.format(input = input_prompt_)

        responese = generalize_llm(prompt)

        print(f'Generalized Task：{responese}')

        return responese
        

    def _decide(self, input_prompt_, tools_):
        print("Determine if you should make a tool.")
        
        decide_llm = OpenAI(temperature=0,model_name = self.model_name)

        description_list = ''
        for tool_ in tools_:
            description = tool_.description
            description_list += description + ','
        description_list = '[' + description_list + ']'

        prompt = """
        You are an agent that determines if the input task is executable. 
        All you can do is to be included in {exec_list}. 
        Answer "Yes." if you can perform the task, or "No." if you cannot.
        
        -----------
        The entered task is:{input}
        -----------
        """.format(exec_list = description_list, input = input_prompt_)

        responese = decide_llm(prompt)

        if responese == "Yes.":
            print('Execute it as it is executable without creating a tool.')

        elif responese == "No.":
            print('It is necessary to create a tool, so run it after creating the tool.')


        return responese


    def _tool_make(self, input_prompt_, folder_path_):
        print("Create a TOOL.")

        tool_llm = OpenAI(temperature=0, model_name = self.model_name)


        create_prompt = """
        Create a python class that can execute {input} with a single string as input.
        Output should be code only.
        The following code was created with the input "multiply two numbers". Please create it like this code.
        Do not enclose the output in ``python ``` or the like.
        
        ------------------
        from dataclasses import dataclass

        @dataclass
        class MultiplicationTool(BaseTool):
            name = "MultiplicationTool"
            description = "used for multiplication. The input is two numbers. For example, if you want to multiply 1 by 2, the input is '1,2'."

            def _run(self, query: str) -> str:
                "Use the tool."
                a, b = query.split(",")
                c = int(a) * int(b)
                result = c

            return result 

            async def _arun(self, query: str) -> str:
                "Use the tool asynchronously."
                raise NotImplementedError("BingSearchRun does not support async")
        ------------------
        """.format(input = input_prompt_)

        code = tool_llm(create_prompt)
        #print('Created Code：\n' + code)



        name_prompt = """
        Get the name of the class from the following code.
        Output is name only.

        ------------------
        {code}
        ------------------
        """.format(code = code)

        name = tool_llm(name_prompt)
        print('Created tool name：' + name)

        # Save to File
        if folder_path_ != None:
            with open(folder_path_ + f'{name}.py', mode='w') as file:
                file.write(code)
        

        return name, code


    def _execute(self,input_prompt_, tools_):
        print("tool to execute it.")

        excute_llm = OpenAI(temperature=0, model_name = self.model_name)

        agent = initialize_agent(tools_, excute_llm, agent="zero-shot-react-description", verbose=True)

        agent.run(input_prompt_)

        return


    def run(self, input_prompt_, tools_, folder_path_ = None):

        generalized_task = self._generalize(input_prompt_)
    
        output = self._decide(generalized_task, tools_)

        if output == "Yes.":
            self._execute(input_prompt_, tools_)

        elif output == "No.":
            new_tool_name, new_tool_code = self._tool_make(generalized_task, folder_path_)

            exec(new_tool_code)

            exec(f'new_tool = {new_tool_name}()')

            tools_.append(new_tool)

            print(tools_)
            
            self._execute(input_prompt_, tools_)
        
        return 

In [108]:
input = 'Sum of prime numbers from 1 to 50'

folder_path = '/home/langchain-tools/data/'
tools = []

lctagi = LCTAGI()

lctagi.run(input, tools, folder_path)

#lctagi.run(input, tools)

Generalized Task：Sum of prime numbers from A to B
Determine if you should make a tool.
It is necessary to create a tool, so run it after creating the tool.
Create a TOOL.


KeyboardInterrupt: 

In [137]:
code = """
class MultiplicationTool(BaseTool):
    name = "MultiplicationTool"
    description = "used for multiplication. The input is two numbers. For example, if you want to multiply 1 by 2, the input is '1,2'."

    def _run(self, query: str) -> str:
      "Use the tool."
      a, b = query.split(",")
      c = int(a) * int(b)
      result = c

      return result 

    async def _arun(self, query: str) -> str:
        "Use the tool asynchronously."
        raise NotImplementedError("BingSearchRun does not support async")

"""

In [138]:
exec(code)

In [133]:
a = MultiplicationTool()
a

MultiplicationTool()

In [135]:
tool_code = """
class NewTool(BaseTool):
    name = "SumOfPrimesTool"
    description = "used for calculating the sum of prime numbers from 1 to 50. The input is a single string."

    def _run(self, query: str) -> str:
        "Use the tool."
        def is_prime(num):
            if num < 2:
                return False
            for i in range(2, num):
                if num % i == 0:
                    return False
            return True

        prime_sum = sum(filter(is_prime, range(1, 51)))
        result = prime_sum

        return result

    async def _arun(self, query: str) -> str:
        "Use the tool asynchronously."
        raise NotImplementedError("SumOfPrimesTool does not support async")

"""

In [136]:
exec(tool_code)

new_tool = NewTool()
new_tool

NewTool(name='SumOfPrimesTool', description='used for calculating the sum of prime numbers from 1 to 50. The input is a single string.', args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0xffff71f1b010>)

In [139]:
exec(code)

a = MultiplicationTool()

tools = []
tools.append(a)

exec(tool_code)

b = NewTool()

tools.append(b)

tools

[MultiplicationTool(name='MultiplicationTool', description="used for multiplication. The input is two numbers. For example, if you want to multiply 1 by 2, the input is '1,2'.", args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0xffff71f1b010>),
 NewTool(name='SumOfPrimesTool', description='used for calculating the sum of prime numbers from 1 to 50. The input is a single string.', args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0xffff71f1b010>)]

In [117]:
llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

agent.run('38 times 47?')

AttributeError: 'MultiplicationTool' object has no attribute 'description'

In [ ]:
exec(code)
            
new_tool = NewTool()
            
tools_.append(new_tool)

In [92]:
tools = []

new_tool_name = 'NewTool'

exec(f'b_tool = {new_tool_name}()')

tools.append(b_tool)

tools

[NewTool(name='SumOfPrimesTool', description='used for calculating the sum of prime numbers from 1 to 50. The input is a single string.', args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0xffff71f1b010>)]

In [84]:
exec(tool_code)



In [ ]:
exec(f'tools_.append({tool_name}())')

In [80]:
tools = []

exec(f'tools.append(NewTool())')

print(tools)

[NewTool(name='SumOfPrimesTool', description='used for calculating the sum of prime numbers from 1 to 50. The input is a single string.', args_schema=None, return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0xffff71f1b010>)]


In [66]:
tools = [NewTool()]

#tools = [MultiplicationTool()]

llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [67]:
input = 'Sum of prime numbers from 1 to 50'

agent.run(input)



> Entering new AgentExecutor chain...
 I need to find the sum of prime numbers from 1 to 50
Action: SumOfPrimesTool
Action Input: "1 to 50"
Observation: 328
Thought: I now know the sum of prime numbers from 1 to 50
Final Answer: 328

> Finished chain.


'328'